In [1]:
#Libreries
import cv2 as cv #Libreria opencv para vision por computador
import matplotlib.pyplot as plt #Libreria para graficacion de resultados
import pandas as pd #Libreria para manejo de datos Dataframe
import numpy as np #Libreria para operaciones aritmeticas avanzadas
from tqdm import tqdm #Libreria para contabilizar tiempo de ejecucion 
import os #Libreria para ejecutar comandos de sistema operativo
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets #Libreria para ejecutar funciones interativas
import statistics as stats

In [2]:
#Lectura de Trayectorias
def read_trajectories(path):
    Trayectories = pd.read_csv(path, header=None)
    Encabezado = Trayectories[0][0]
    Trayectories = Trayectories.drop([0],axis=0) 
    return Trayectories,Encabezado

In [3]:
#Filter
def first_filter_trayectories(path):
    Trayectories = pd.read_csv(path,header=None)
    Encabezado = Trayectories[0][0] #Extraigo el encabezado
    Trayectories = Trayectories.rename(columns = {0:'tray'}) #Cambio el nombre de la columna por uno mas intuitivo
    Trayectories = Trayectories.drop([0],axis=0) #Elimino la fila perteneciente al encabezado.
    Encabezado = Encabezado.split(' ')
    print("Numero total de trayectorias calculadas:",Encabezado[0])
    print("Largo del video:",Encabezado[1])
    print("Ancho del video:",Encabezado[2])
    print("Numero total de frames del video:",Encabezado[3])
    print("-----------------------------------------------")
    TF = pd.DataFrame(columns=('tray',))
    for i in tqdm(Trayectories['tray']):
        t = i.split(' ')
        d = np.sqrt((float(t[2])-float(t[-3]))**2 + (float(t[3])-float(t[-2]))**2)
        if(d>5):
            #Obtengo el dataframe filtrado
            TF.loc[len(TF)]=[i,]
    ##Retorna el encabezado antiguo ya q lo necesitamos en la funcion guardar cabecera
    Encabezado[0] = len(TF)
    return TF,Encabezado

In [4]:
#Guardo las trayectorias filtradas
def save_trajectories(TF,ruta_txt):
    TrayectoriasF = open(ruta_txt,'w')
    for i in tqdm(TF['tray']):
        TrayectoriasF.write(i+"\n")
    TrayectoriasF.close()

In [5]:
#Para el desarrollo de las graficas si es necesario que guardes la información de la cabecera en el txt
def save_header(TF,Encabezado,ruta):
    CabeceraF = open(ruta,'w')
    CabeceraF.write(str(TF.shape[0])+" "+Encabezado[1]+" "+Encabezado[2]+" "+Encabezado[3])
    CabeceraF.close()

In [6]:
#Funcion para un nuevo filtro(Filtrar por area)
def second_filter_trajectories(rutatxt, ruta_cabecera):
    Trayectories = pd.read_csv(rutatxt,header=None)
    Encabezado = pd.read_csv(ruta_cabecera, header=None)
    Encabezado = Encabezado.split(' ')
    print("Numero total de trayectorias calculadas:",Encabezado[0])
    print("Largo del video:",Encabezado[1])
    print("Ancho del video:",Encabezado[2])
    print("Numero total de frames del video:",Encabezado[3])
    print("-----------------------------------------------")
    TF = pd.DataFrame(columns=(0,))
    for i in tqdm(Trayectories[0]):
        t = i.split(' ')
        d = np.sqrt((float(t[2])-float(t[-3]))**2 + (float(t[3])-float(t[-2]))**2)
        x=2
        y=3 
        good = True
        for _ in range(16):
            if(d<4 or (float(t[y])<5 or float(t[y])>245 or float(t[x])<10 or float(t[x])>125)):
                    good = False
                    break
            x+=2
            y+=2
        if(good == True):
            TF.loc[len(TF)]=[i,]
    Encabezado[0] = len(TF)
    return TF,Encabezado

In [7]:
#Calculo de las medianas de un dataset:
def Medianas_t(rutatxt):
    #La Funcion sirve solo para dataset filtrados
    Trayectories = pd.read_csv(rutatxt,header=None)
    Trayectories = Trayectories.rename(columns = {0:'tray'})
    medianas = pd.DataFrame(columns=('tray',))
    t1 = Trayectories['tray'][0].split(' ')
    f1 = t1[0]
    f2 = t1[1]
    x=[] #lista que contendra las posiciones en x
    y=[] #lista que contendra las posiciones en y
    for i in Trayectories['tray']:
        t = i.split(" ")
        px=2
        py=3
        if(f1 == t[0] and f2 == t[1]):
            for _ in range(16):
                x.append(float(t[px]))
                y.append(float(t[py]))
                px+=2
                py+=2
        else:
            medianax = stats.median(x)
            medianay = stats.median(y)
            x=[]
            y=[]
            medianas.loc[len(medianas)]=[f1+' '+f2+' '+str(medianax)+' '+str(medianay),]
            f1 = t[0]
            f2 = t[1]
            px=2
            py=3
            for _ in range(16):
                x.append(float(t[px]))
                y.append(float(t[py]))
                px+=2
                py+=2
    return medianas

In [10]:
def FiltrarporMedianas(rutatxt,medianas,path_header):
    Trayectories = pd.read_csv(rutatxt,header=None)
    Encabezado = pd.read_csv(path_header,header=None)
    Encabezado = Encabezado.split(' ')
    TF = pd.DataFrame(columns=(0,))
    index = 0
    median = medianas[0][index].split(' ')
    for i in tqdm(Trayectories[0]):
        t = i.split(" ")
        if(t[0] == median[0] and t[1] == median[1]):
            px = (float(t[2])+float(t[4])+float(t[6])+float(t[8])+float(t[10])+float(t[12])+float(t[14])+float(t[16])+float(t[18])+float(t[20])+float(t[22])+float(t[24])+float(t[26])+float(t[28])+float(t[30])+float(t[32]))/16
            py = (float(t[3])+float(t[5])+float(t[7])+float(t[9])+float(t[11])+float(t[13])+float(t[15])+float(t[17])+float(t[19])+float(t[21])+float(t[23])+float(t[25])+float(t[27])+float(t[29])+float(t[31])+float(t[33]))/16
            d = np.sqrt((px-float(median[2]))**2 + (py-float(median[3]))**2)
            if(d<10):
                TF.loc[len(TF)]=[i,]
        else:
            if(index == (medianas.shape[0]-1)):
                break
            index+=1
            median = medianas[0][index].split(' ')
            px = (float(t[2])+float(t[4])+float(t[6])+float(t[8])+float(t[10])+float(t[12])+float(t[14])+float(t[16])+float(t[18])+float(t[20])+float(t[22])+float(t[24])+float(t[26])+float(t[28])+float(t[30])+float(t[32]))/16
            py = (float(t[3])+float(t[5])+float(t[7])+float(t[9])+float(t[11])+float(t[13])+float(t[15])+float(t[17])+float(t[19])+float(t[21])+float(t[23])+float(t[25])+float(t[27])+float(t[29])+float(t[31])+float(t[33]))/16
            d = np.sqrt((px-float(median[2]))**2 + (py-float(median[3]))**2)
            if(d<10):
                TF.loc[len(TF)]=[i,]
    Encabezado[0] = len(TF)
    return TF,Encabezado